In [1]:
%load_ext autoreload
# Enable autoreload for all modules
%autoreload 2

In [2]:
from iris_environments.environments import get_environment_builder, get_robot_instance_names
from cspace_utils.graphs import compute_greedy_clique_partition
from cspace_utils.plotting import (plot_cliques, 
                                   plot_visibility_graph, 
                                   plot_edges, 
                                   plot_points,
                                   plot_2d_ellipse_meshcat,
                                   plot_vpoly_2d_meshcat)
from cspace_utils.environment_helpers import get_col_func, get_coverage_estimator, get_sample_cfree_handle, vgraph
import numpy as np
import ipywidgets as widgets
from functools import partial
from pydrake.all import (RigidTransform, Rgba, Sphere, RotationMatrix, SceneGraphCollisionChecker)

In [3]:
from iris_environments.environments import env_names
currname = '2DOFBLOCKS'
plant_builder = get_environment_builder(currname)
plant, scene_graph, diagram, diagram_context, plant_context, meshcat = plant_builder(usemeshcat=True)

scene_graph_context = scene_graph.GetMyMutableContextFromRoot(
    diagram_context)

rob_names = get_robot_instance_names(currname)
robot_instances = [plant.GetModelInstanceByName(n) for n in rob_names]
checker = SceneGraphCollisionChecker(model = diagram, 
                    robot_model_instances = robot_instances,
                    #configuration_distance_function = _configuration_distance,
                    edge_step_size = 0.125)

INFO:drake:Meshcat listening for connections at http://localhost:7000
INFO:drake:Allocating contexts to support implicit context parallelism 20


http://localhost:7000


In [4]:
col_hand=get_col_func(plant, plant_context, use_ik=False)
sample_cfree = get_sample_cfree_handle(plant.GetPositionLowerLimits(), 
                        plant.GetPositionUpperLimits(),
                        col_hand)

In [5]:
#pts,_ = sample_cfree(50, 1000,[])
xw = 0.2
yw = 0.2
z = 0

samples = []
for _ in range(10):
    s = np.array([xw, yw])*2*(np.random.rand(2)-0.5)
    samples.append(s)

samples.append([-0.4, 0.29])
samples.append([-0.4, -0.29])
samples.append([0.4, 0.29])
samples.append([0.4, -0.29])
pts = np.array(samples)
plot_points(meshcat, pts, 'vgraph/points', size=0.02)
ad_mat = vgraph(pts, checker=checker, parallelize=True)

In [6]:
from pydrake.all import Hyperellipsoid, VPolytope

cliques = compute_greedy_clique_partition(ad_mat.toarray(), min_cliuqe_size=10)
colors = [Rgba(0,0.1,0,1)]
plot_cliques(meshcat, cliques, pts, 'cliques', downsampling=5, size = 0.001, colors= colors)

clique_pts = [pts[c] for c in cliques]
ellipsoids = [Hyperellipsoid.MinimumVolumeCircumscribedEllipsoid(p.T) for p in clique_pts if len(p)>4]
for i, e in enumerate(ellipsoids):
    plot_2d_ellipse_meshcat(meshcat, e, f"cliques/ellipsoids/e_{i}", size =0.002)
  

vpolys = [VPolytope(c.T).GetMinimalRepresentation() for c in clique_pts]
for i, v in enumerate(vpolys):
    plot_vpoly_2d_meshcat(meshcat, v, f"cliques/vpolys/v_{i}", size =0.005, color= Rgba(0,0.3,0.8, 1))

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-11


In [7]:
from pydrake.all import FastIris, FastIrisOptions, HPolyhedron
meshcat.Delete("/drake/iris")

fio = FastIrisOptions()
fio.num_particles = 400
fio.target_uncertainty = 0.1
fio.admissible_proportion_in_collision = 0.01
fio.max_iterations = 1
for i, e in enumerate(ellipsoids):
    region = FastIris(checker, e,HPolyhedron.MakeBox([-3,-3], [3,3]), fio)
    region_vpoly = VPolytope(region)
    plot_vpoly_2d_meshcat(meshcat, region_vpoly, f"iris/vpolys/v_{i}", size =0.01, color= Rgba(0.7,0.0,0.7, 1))


INFO:drake:FastIris requires 399/400 particles to be collision free 
INFO:drake:FastIris iteration 0
INFO:drake:SeparatingPlanes iteration: 1 faces: 8


INFO:drake:FastIris iter 1, iter limit 1
INFO:drake:Fast Iris execution time : 52 ms


In [8]:
fio = FastIrisOptions()
fio.num_particles = 400
fio.target_uncertainty = 0.1
fio.admissible_proportion_in_collision = 0.01
fio.max_iterations = 1

for i, e in enumerate(ellipsoids):
    fio.containment_points = clique_pts[i].T
    fio.force_containment_points = True
    region = FastIris(checker, e,HPolyhedron.MakeBox(plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()), fio)
    region_vpoly = VPolytope(region)
    plot_vpoly_2d_meshcat(meshcat, region_vpoly, f"iris/vpolys_col_free/v_{i}", size =0.01, color= Rgba(0.0,0.4,0.0, 1))

INFO:drake:FastIris requires 399/400 particles to be collision free 
INFO:drake:FastIris iteration 0


INFO:drake:FastIris Warning relaxing cspace margin by 0.008054168709700438 to ensure point containment
INFO:drake:SeparatingPlanes iteration: 1 faces: 6
INFO:drake:FastIris iter 1, iter limit 1
INFO:drake:Fast Iris execution time : 65 ms


In [11]:
vp = vpolys[0]

In [ ]:
vp.

In [5]:
#make configuration file 
# import os
# point_file = 'benchmarks/visibility_graphs/'+currname+'_pts.yml'
# if point_file.split('/')[2] in os.listdir('benchmarks/seedpoints'):
#     file = open(point_file, 'a') 
# else:
#     file = open(point_file, 'w')
#     file.write('points:\n')

FileNotFoundError: [Errno 2] No such file or directory: 'benchmarks/visibility_graphs/2DOFBLOCKS_pts.yml'

In [ ]:
q = np.zeros(plant.num_positions()) 
sliders = []
for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]*0.99
    q_high = plant.GetPositionUpperLimits()[i]*0.99
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, step=0.001, description=f"q{i}"))

col_col =  Rgba(0.8, 0.0, 0, 0.5)    
col_free =  Rgba(0.0, 0.8, 0.5, 0.5) 
def showres(qvis):
    plant.SetPositions(plant_context, qvis)
    diagram.ForcedPublish(diagram_context)
    query = plant.get_geometry_query_input_port().Eval(plant_context)
    col = query.HasCollisions()
    if col:
        meshcat.SetObject(f"/drake/visualizer/shunk",
                                   Sphere(0.2),
                                   col_col)
    else:
        meshcat.SetObject(f"/drake/visualizer/shunk",
                                   Sphere(0.2),
                                   col_free)
    meshcat.SetTransform(f"/drake/visualizer/shunk",
                                   RigidTransform(RotationMatrix(),
                                                  np.array([0,0,2])))
    return col

def handle_slider_change(change, idx):
    q[idx] = change['new']
    showres(q)
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

def write_seed_point_to_file(button):
    col = showres(q)
    if not col:
        line  = '- ['
        for a in q[:-1]:
            line+= str(a)+', '
        line+= str(q[-1])+']\n'
        file.write(line)
        file.flush()
        #write seedpoint to file
    else:
        raise ValueError("That point is in collision")

button = widgets.Button(description="Save")

# Attach the function to the button's click event
button.on_click(write_seed_point_to_file)

# Display the button
display(button)